In [1]:
from unityagents import UnityEnvironment
import numpy as np

from maddpg_agent import Agent
import torch
from collections import deque
from matplotlib import pyplot as plt

%matplotlib inline


The device to use:  cuda:0


In [2]:
env = UnityEnvironment(file_name="./Tennis_Linux/Tennis.x86_64")

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=False)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])
print('The state for the second agent looks like:', states[1])

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 
Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]
The state for the sec

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("The device to use: ", device)

agents =[] 

for i in range(num_agents):
    agents.append(Agent(state_size, action_size, random_seed=0))
    
Agent.instances[0].actor_local.load_state_dict(torch.load('checkpoint_actor0.pth'))
Agent.instances[1].actor_local.load_state_dict(torch.load('checkpoint_actor1.pth'))

scores = np.zeros(num_agents)
while True:
    actions = np.array([agents[i].act(states[i]) for i in range(num_agents)])

    env_info = env.step(actions)[brain_name]        # send the action to the environment
    next_states = env_info.vector_observations     # get the next state
    rewards = env_info.rewards                     # get the reward
    dones = env_info.local_done        

    states = next_states
    scores += rewards

    print('\rScores: {:.2f}\t{:.2f}'
              .format(scores[0], scores[1]), end="") 
    
    if np.any(dones):
        break
        
print("\nScores: {}".format(scores))

The device to use:  cuda:0
Initialising ReplayBuffer
Appended to Agent.instances agent 0
Appended to Agent.instances agent 1
Scores: 2.50	2.60
Scores: [2.50000004 2.60000004]


In [4]:
env.close()